<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [25]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [26]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);

The delimiter in pedigree.txt is ','.
Finished!


In [27]:
first(phenotypes,5)

,ID,y1,y2,y3,x1,x2,x3,dam
,String,Float64⍰,Float64⍰,Float64⍰,Float64,Float64,String,String
1,a1,-0.06,3.58,-1.18,0.9,2.0,m,0
2,a3,-2.07,3.19,missing,0.7,2.0,f,0
3,a4,-2.63,6.97,-0.83,0.6,1.0,m,a2
4,a5,2.31,missing,-1.52,0.4,2.0,m,a2
5,a6,0.93,4.87,-0.01,5.0,2.0,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [28]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [29]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [30]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [31]:
G1 = 1.0
G2 = [1.0 0.5
      0.5 1.0]
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6:Run Analysis</button> 

In [32]:
out1=runMCMC(model1,phenotypes,chain_length=5000,output_samples_frequency=100);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Phenotypes for all traits included in the model for individual a7 in the row 6 are missing. This record is deleted

A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12

MCMC Information:

methods                   conventional (no markers)
chain_length                                   5000
burnin                                            0
estimateScale                                 false
starting_value                                false
printout_freque

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00




The version of Julia and Platform in use:

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [33]:
keys(out1)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "EBV_y1"
  "heritability"
  "location parameters"
  "residual variance"
  "polygenic effects covariance matrix"
  "genetic_variance"

In [34]:
out1["polygenic effects covariance matrix"]

,Covariance,Estimate,Std_Error
,Any,Any,Any
1,y1:ID_y1:ID,3.0265,4.42612
2,y1:ID_y1:dam,1.68589,2.69372
3,y1:dam_y1:ID,1.68589,2.69372
4,y1:dam_y1:dam,2.63614,3.64146


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [35]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [36]:
R      = [1.0 0.5 0.5
          0.5 1.0 0.5
          0.5 0.5 1.0]
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [37]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [38]:
G1 = [1.0 0.5
      0.5 1.0]
G2 = [1.0 0.5 0.5 0.5
      0.5 1.0 0.5 0.5
      0.5 0.5 1.0 0.5
      0.5 0.5 0.5 1.0]
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

x2 is not found in model equation 1.
dam is not found in model equation 2.
dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Run Analysis</button> 

In [39]:
out2=runMCMC(model2,phenotypes,chain_length=5000,output_samples_frequency=100,outputEBV=true);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Phenotypes for all traits included in the model for individual a7 in the row 6 are missing. This record is deleted

A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2

MCMC Information:

methods                   conventional (no markers)
chain_length                                   5000
burnin                                            0
es

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01




The version of Julia and Platform in use:

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [40]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 8 entries. Keys:
  "EBV_y2"
  "EBV_y1"
  "heritability"
  "location parameters"
  "residual variance"
  "polygenic effects covariance matrix"
  "EBV_y3"
  "genetic_variance"

In [41]:
out2["location parameters"]

,Trait,Effect,Level,Estimate,Std_Error
,Any,Any,Any,Any,Any
1,y1,intercept,intercept,-48.4832,missing
2,y1,x1,x1,0.656452,missing
3,y1,x3,m,48.0757,missing
4,y1,x3,f,46.1323,missing
5,y1,ID,a2,-0.0453471,missing
6,y1,ID,a1,-0.134154,missing
7,y1,ID,a3,-0.0823497,missing
8,y1,ID,a7,-0.161346,missing
9,y1,ID,a4,-0.724968,missing


In [42]:
out2["EBV_y3"]

,ID,EBV,PEV
,Any,Any,Any
1,a2,0.161721,0.929215
2,a1,-0.291457,0.673189
3,a3,-0.0900826,0.755317
4,a7,-0.0987875,0.923473
5,a4,-0.175539,0.78755
6,a6,-0.0912501,0.787073
7,a9,-0.0221752,1.1382
8,a5,0.178192,0.842206
9,a10,-0.0803942,0.55782


In [43]:
out2["polygenic effects covariance matrix"]

,Covariance,Estimate,Std_Error
,Any,Any,Any
1,y1:ID_y1:ID,1.20443,missing
2,y1:ID_y2:ID,0.468268,missing
3,y1:ID_y3:ID,0.50878,missing
4,y1:ID_y1:dam,0.498071,missing
5,y2:ID_y1:ID,0.468268,missing
6,y2:ID_y2:ID,1.06968,missing
7,y2:ID_y3:ID,0.490239,missing
8,y2:ID_y1:dam,0.477436,missing
9,y3:ID_y1:ID,0.50878,missing


In [44]:
out2["genetic_variance"]

,Covariance,Estimate,Std_Error
,Any,Any,Any
1,y1_y1,1.98713,1.8368
2,y1_y2,0.540547,0.901615
3,y1_y3,0.468791,0.725382
4,y2_y1,0.540547,0.901615
5,y2_y2,0.623483,0.511865
6,y2_y3,0.237568,0.333816
7,y3_y1,0.468791,0.725382
8,y3_y2,0.237568,0.333816
9,y3_y3,0.61414,0.673545
